In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import joblib

# ====== Definir Modelo ======
class LinkMLP(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

# ====== Carregar modelo ======
model = LinkMLP()
model.load_state_dict(torch.load("../models/mlp.pth", weights_only=True))  # só os pesos
model.eval()

scaler = joblib.load("../models/scaler.pkl")

# ====== Dados de teste ======
test_data = [
    [-55, 2, 17, 0.3],  # [rssi, etx, delay, busy_fraction]
    [-85, 3.8, 70, 0.1] # [rssi, etx, delay, busy_fraction]
]

X_test = torch.tensor(scaler.transform(test_data), dtype=torch.float)

# ====== Inferência ======
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(test_data):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"Aresta {row} -> Classe {cls} ({CLASS_NAMES[cls]}) (confiança {conf:.2f})")

Aresta [-55, 2, 17, 0.3] -> Classe 1 (Bom) (confiança 0.99)
Aresta [-85, 3.8, 70, 0.1] -> Classe 0 (Ruim) (confiança 0.99)
